In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA




df = pd.read_csv('marketing_campaign.csv', delimiter=';')
"""
print("Dataset shape:", df.shape)
print("Columns:", df.columns.tolist())
print("\nFirst 5 rows:")
print(df.head())
print("\nMissing values per column:")
print(df.isnull().sum())
print("\nSummary statistics:")
print(df.describe())
print("\nData types:")
print(df.dtypes)
"""

income_median = df['Income'].median()
df['Income'] = df['Income'].fillna(income_median)

#datetime
df['Dt_Customer'] = pd.to_datetime(df['Dt_Customer'], format='%d-%m-%Y')

#(assuming today is max Dt_customer+1 day)
max_date = df['Dt_Customer'].max()
df['Customer_Tenure_Days'] = (max_date - df['Dt_Customer']).dt.days

# Create Age
df['Age'] = 2025 - df['Year_Birth']  # use current year or any year of analysis

# quick check
print(df[['Income', 'Dt_Customer', 'Customer_Tenure_Days', 'Age']].head())


df.drop(['Z_CostContact', 'Z_Revenue'], axis=1, inplace=True)
df.drop(columns=[col for col in ['Z_CostContact', 'Z_Revenue'] if col in df.columns], inplace=True)
print(df.columns.tolist())

#add coloumns
df['Children'] = df['Kidhome'] + df['Teenhome']

df['Age'] = 2025 - df['Year_Birth']
df['Dt_Customer'] = pd.to_datetime(df['Dt_Customer'], format='%d-%m-%Y')
max_date = df['Dt_Customer'].max()
df['Customer_Tenure_Days'] = (max_date - df['Dt_Customer']).dt.days
df['Total_Spent'] = df[['MntWines', 'MntFruits', 'MntMeatProducts',
                        'MntFishProducts', 'MntSweetProducts',
                        'MntGoldProds']].sum(axis=1)

df['Total_Campaigns_Accepted'] = df[['AcceptedCmp1', 'AcceptedCmp2',
                                      'AcceptedCmp3', 'AcceptedCmp4',
                                      'AcceptedCmp5']].sum(axis=1)






#DATA VISUALIZATION
plt.figure(figsize=(8,5))
sns.scatterplot(data=df, x='Age', y='Income')
plt.title('Age vs. Income')
plt.show()


plt.figure(figsize=(6,4))
sns.countplot(data=df, x='Children')
plt.title('Number of Children in Household')
plt.show()


plt.figure(figsize=(7,4))
sns.countplot(data=df, x='Marital_Status', order=df['Marital_Status'].value_counts().index)
plt.title('Marital Status Distribution')
plt.xticks(rotation=45)
plt.show()


plt.figure(figsize=(8,5))
sns.histplot(df['Income'], bins=30, kde=True)
plt.title('Income Distribution')
plt.xlabel('Income')
plt.ylabel('Number of Customers')
plt.show()


plt.figure(figsize=(8,5))
sns.histplot(df['Age'], bins=30, kde=True)
plt.title('Age Distribution')
plt.xlabel('Age')
plt.ylabel('Number of Customers')
plt.show()
"""
"""
plt.figure(figsize=(8,5))
sns.histplot(df['Customer_Tenure_Days'], bins=30, kde=True)
plt.title('Customer Tenure Distribution')
plt.xlabel('Tenure (Days)')
plt.show()

plt.figure(figsize=(8,5))
sns.histplot(df['Total_Spent'], bins=30, kde=True)
plt.title('Total Spent Distribution')
plt.xlabel('Total Spent')
plt.show()

plt.figure(figsize=(8,5))
sns.histplot(df['Total_Campaigns_Accepted'], bins=30, kde=True)
plt.title('Total Campaigns Accepted Distribution')
plt.xlabel('Total Campaigns Accepted')
plt.show()


features = ['Age', 'Income', 'Children', 'Total_Spent', 'Customer_Tenure_Days',
            'Total_Campaigns_Accepted']

# drop rows with missing values in features
X = df[features].dropna()

#scale
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

inertia = []
K_range = range(1, 11)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X_scaled)
    inertia.append(kmeans.inertia_)

plt.figure(figsize=(8,4))
plt.plot(K_range, inertia, marker='o')
plt.xlabel('Number of clusters (k)')
plt.ylabel('Inertia (Sum of squared distances)')
plt.title('Elbow Method For Optimal k')
plt.show()

optimal_k = 4  # example, adjust based on elbow plot
kmeans = KMeans(n_clusters=optimal_k, random_state=42)
df['Cluster'] = kmeans.fit_predict(X_scaled)

cluster_profile = df.groupby('Cluster')[features].mean()
print(cluster_profile)
sns.pairplot(df, vars=features, hue='Cluster', palette='tab10')
plt.show()



ValueError: time data "2012-09-04" doesn't match format "%d-%m-%Y", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.